# Kerasによるクレーム分類：Pythonディープラーニングライブラリ補講
## SimpleRNN/GRU/LSTMの比較

このノートブックでは、複数のテキスト分類アルゴリズムを使ってモデルの精度や学習時間、推論速度の比較をすることができます。Kerasで簡単に試すことができるアルゴリズムとして、LSTM のほかに SimpleRNN と GRU があります。

このノートブックでは、以下で構成されるテキスト分析プロセスについて説明します。

- グローブ語の埋め込みを使用した例の類推
- GloVe単語の埋め込みを使用したトレーニングデータのベクトル化
- SimpleRNN/GRU/LSTMベースの分類モデルの作成とトレーニング
- モデルを使用して分類を予測する

## モジュールを準備する

このノートブックは、Kerasライブラリを使用して分類器を作成およびトレーニングします。

In [ ]:
import string
import re
import os
import numpy as np
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
import keras
from keras import models, layers, optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, GRU, SimpleRNN
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tf.__version__)


**事前トレーニング済みのGloVe単語の埋め込みをダウンロードして、このノートブックにロードしてみましょう。**

これで、サイズが **400,000** ワードの `dictionary`と、辞書内の単語に対応する ` GloVe wordベクトル ` が作成されます。 各単語ベクトルのサイズは50であるため、ここで使用される単語埋め込みの次元は **50** です。

*次のセルの実行には数分かかる場合があります*

In [ ]:
words_list_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                  'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordsList.npy')

word_vectors_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordVectors.npy')

word_vectors_dir = './word_vectors'

if not (os.path.exists(os.path.join(word_vectors_dir, 'wordsList.npy'))):
    os.makedirs(word_vectors_dir, exist_ok=True)
    urllib.request.urlretrieve(words_list_url, os.path.join(word_vectors_dir, 'wordsList.npy'))
else:
    dictionary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
    dictionary = dictionary.tolist()
    dictionary = [word.decode('UTF-8') for word in dictionary]
    print('Loaded the dictionary! Dictionary size: ', len(dictionary))

if not (os.path.exists(os.path.join(word_vectors_dir, 'wordVectors.npy'))):
    urllib.request.urlretrieve(word_vectors_url, os.path.join(word_vectors_dir, 'wordVectors.npy'))
else:
    word_vectors = np.load(os.path.join(word_vectors_dir, 'wordVectors.npy'))
    print ('Loaded the word vectors! Shape of the word vectors: ', word_vectors.shape)

**収縮マップという単語を作成します。 マップはコーパスの縮約を拡張するために使用されます（たとえば、 "can't" は"cannot"になります）。**

In [ ]:
contractions_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/contractions.xlsx')

if not (os.path.exists('./contractions/contractions.xlsx')):
    contractions_df = pd.read_excel(contractions_url)
else:
    contractions_df = pd.read_excel('./contractions/contractions.xlsx')
    
contractions = dict(zip(contractions_df.original, contractions_df.expanded))
print('Review first 10 entries from the contractions map')
print(contractions_df.head(10))

## トレーニングデータを準備する

Contoso Ltdは、クレームテキストとして受け取るテキストの例を含む小さなドキュメントを提供しています。 彼らは、これをサンプルクレームごとに1行のテキストファイルで提供しています。

次のセルを実行して、ファイルの内容をダウンロードして確認します。 少し時間をかけてクレームを読んでください（あなたはそれらのいくつかをかなりコミカルに感じるかもしれません！）。

In [ ]:
data_location = './data'
base_data_url = 'https://databricksdemostore.blob.core.windows.net/data/05.03/'
filesToDownload = ['claims_text.txt', 'claims_labels.txt']

claims_corpus = [claim for claim in open(os.path.join(data_location, 'claims_text.txt'))]
claims_corpus

クレームサンプルに加えて、Contoso Ltdは、提供されたサンプルクレームのそれぞれに0（「住宅保険クレーム」）または1（「自動車保険クレーム」）のいずれかとしてラベルを付けるドキュメントも提供しています。 これは、サンプルごとに1行のテキストファイルとして提示され、クレームテキストと同じ順序で提示されます。

次のセルを実行して、提供されたClaims_labels.txtファイルの内容を調べます。

In [ ]:
labels = [int(re.sub("\n", "", label)) for label in open(os.path.join(data_location, 'claims_labels.txt'))]
print(len(labels))
print(labels[0:5]) # first 5 labels
print(labels[-5:]) # last 5 labels

上記の出力からわかるように、値は整数0または1です。これらをモデルのトレーニングに使用するラベルとして使用するには、これらの整数値をカテゴリ値に変換する必要があります（他のプログラミングの列挙型のようなものと考えてください） 言語）。

`keras.utils` の to_categorical メソッドを使用して、これらの値をバイナリのカテゴリ値に変換できます。 次のセルを実行します。

In [ ]:
labels = to_categorical(labels, 2)
print(labels.shape)
print()
print(labels[0:2]) # first 2 categorical labels
print()
print(labels[-2:]) # last 2 categorical labels

クレームテキストとラベルが読み込まれたので、テキスト分析プロセスの最初のステップ、つまりテキストを正規化する準備ができました。

### クレームコーパスを処理する

- すべての単語を小文字にします
- 収縮を拡張する（たとえば、 "can't" が "cannot" になる）
- 特殊文字（句読点など）を削除する
- クレームテキスト内の単語のリストを、辞書内のそれらの単語の対応するインデックスのリストに変換します。 書面のクレームに表示される単語の順序は維持されることに注意してください。

次のセルを実行して、クレームコーパスを処理します。

In [ ]:
def remove_special_characters(token):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_token = pattern.sub('', token)
    return filtered_token

def convert_to_indices(corpus, dictionary, c_map, unk_word_index = 399999):
    sequences = []
    for i in range(len(corpus)):
        tokens = corpus[i].split()
        sequence = []
        for word in tokens:
            word = word.lower()
            if word in c_map:
                resolved_words = c_map[word].split()
                for resolved_word in resolved_words:
                    try:
                        word_index = dictionary.index(resolved_word)
                        sequence.append(word_index)
                    except ValueError:
                        sequence.append(unk_word_index) #Vector for unkown words
            else:
                try:
                    clean_word = remove_special_characters(word)
                    if len(clean_word) > 0:
                        word_index = dictionary.index(clean_word)
                        sequence.append(word_index)
                except ValueError:
                    sequence.append(unk_word_index) #Vector for unkown words
        sequences.append(sequence)
    return sequences

claims_corpus_indices = convert_to_indices(claims_corpus, dictionary, contractions)

**固定長ベクトルを作成する**

クレームで使用される単語の数は、クレームによって異なります。 固定サイズの入力ベクトルを作成する必要があります。 `keras.preprocessing.sequence` のユーティリティ関数 ` pad_sequences` を使用して、単語インデックスの固定サイズのベクトル（サイズ= 125）を作成します。

In [ ]:
maxSeqLength = 125

X = pad_sequences(claims_corpus_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

print('Review the new fixed size vector for a sample claim')
print(remove_special_characters(claims_corpus[5]).split())
print()
print(X[5])
print('')
print('Lenght of the vector: ', len(X[5]))

## 1. SimpleRNNを構築する

テキストデータのトレーニングから入力特徴を前処理したので、分類器を作成する準備ができました。 この場合、SimpleRNNを構築します。 ネットワークには、単語インデックスをGloVe単語ベクトルに変換する単語埋め込みレイヤーがあります。 次に、GloVeワードベクトルがSimpleRNN層に渡され、その後にバイナリ分類出力層が続きます。

<img src="../media/simplernn.png" >

SimpleRNNはLSTMと比べて回路がかなり単純です。処理中に前の隠れ状態を次の単語ステップに渡します。隠れ状態は、ニューラルネットワークの記憶として機能します。ネットワークが以前に見た以前のデータに関する情報を保持します。回路内部での計算は**tanh**で値の調整くらいしかしていませんが、短い文章ではかなりうまく動作します。

次のセルを実行して、ニューラルネットワークの構造を構築します。

In [ ]:
embedding_layer = Embedding(word_vectors.shape[0],
                            word_vectors.shape[1],
                            weights=[word_vectors],
                            input_length=maxSeqLength,
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))

model.summary()

## ニューラルネットワークをトレーニングする

最初に、データを2つのセットに分割します：（1）トレーニングセットと（2）検証またはテストセット。 検証セットの精度は、モデルのパフォーマンスを測定するために使用されます。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)

 `Adam` 最適化アルゴリズムを使用してモデルをトレーニングします。 また、問題のタイプが `Binary Classification` であるため、出力層には `Sigmoid` アクティベーション関数を使用し、損失関数には `Binary Crossentropy` を使用しています。

In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

これで、トレーニングデータとラベルに合わせてSimpleRNNに学習させる準備ができました。 トレーニングのバッチサイズとエポック数を定義しました。

次のセルを実行して、モデルをデータに合わせます。

In [ ]:
# EaelyStoppingの設定
early_stopping =  EarlyStopping(
                            monitor='val_loss',
                            min_delta=0.0,
                            patience=2
    )

epochs = 100
batch_size = 16
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

SimpleRNNモデルの精度を出力します。

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])


## SimpleRNNで分類クレームのテスト

モデルを構築したので、一連のクレームに対してそれを試してください。 最初にテキストを前処理する必要があることを思い出してください。

次のセルを実行して、テストデータを準備します。

In [ ]:
test_claim = ['I crashed my car into a pole.', 
              'The flood ruined my house.', 
              'I lost control of my car and fell in the river.']

test_claim_indices = convert_to_indices(test_claim, dictionary, contractions)
test_data = pad_sequences(test_claim_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

次に、モデルを使用して分類を予測します。

In [ ]:
pred = model.predict(test_data)
pred_label = pred.argmax(axis=1)
pred_df = pd.DataFrame(np.column_stack((pred,pred_label)), columns=['class_0', 'class_1', 'label'])
pred_df.label = pred_df.label.astype(int)
print('Predictions')
pred_df

これまでのセッションとmodelをクリアします。

In [ ]:
keras.backend.clear_session()

# 2. GRU

GRU(Gated recurrent unit)は新世代の再帰型ニューラルネットワークであり、LSTMによく似ています。GRUはセル状態を取り除き、隠れ状態を使用して情報を転送します。また、リセットゲートと更新ゲートの2つのゲートしかありません。GRUはLSTMに比べ、行列演算が少なくLSTMを学習するよりもいくらか高速です。ただし、精度についてはLSTMとGRUでは明確な差はありませんので、シーンによってどちらも検討するようにしてください。

<img src="../media/gru.png" />


In [ ]:
embedding_layer = Embedding(word_vectors.shape[0],
                            word_vectors.shape[1],
                            weights=[word_vectors],
                            input_length=maxSeqLength,
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)

opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# EaelyStoppingの設定
early_stopping =  EarlyStopping(
                            monitor='val_loss',
                            min_delta=0.0,
                            patience=2
    )

epochs = 100
batch_size = 16
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

GRUモデルの精度を出力します。

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

## GRUで分類クレームのテスト

モデルを構築したので、一連のクレームに対してそれを試してください。 最初にテキストを前処理する必要があることを思い出してください。

次のセルを実行して、テストデータを準備します。

In [ ]:
test_claim = ['I crashed my car into a pole.', 
              'The flood ruined my house.', 
              'I lost control of my car and fell in the river.']

test_claim_indices = convert_to_indices(test_claim, dictionary, contractions)
test_data = pad_sequences(test_claim_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

次に、モデルを使用して分類を予測します。

In [ ]:
pred = model.predict(test_data)
pred_label = pred.argmax(axis=1)
pred_df = pd.DataFrame(np.column_stack((pred,pred_label)), columns=['class_0', 'class_1', 'label'])
pred_df.label = pred_df.label.astype(int)
print('Predictions')
pred_df

これまでのセッションとmodelをクリアします。

In [ ]:
keras.backend.clear_session()

# 3. LSTM

LSTM(Long short-term memory)は再帰型ニューラルネットワークの中で、最も回路が複雑です。LSTMは、セルの状態と、さまざまなゲートから成り立っています。これらは、LSTMが情報を保持または忘れることを可能にするために使用されます。最も大きな特長は、従来のRNNでは学習できなかった長期依存(long-term dependencies)を学習可能であるという点です。SimpleRNNのような構造だと、長い文章を学習することができませんでした（勾配消失問題）。LSTMはこの問題を解決しています。

<img src="../media/lstm.png" />

LSTMは入力として通常の入力と、一つ前の隠れ層からの入力の2つがあります。この2つの入力によって、セルの状態と3つのゲートの値が更新されます。処理のプロセスは以下の通りです。
1. ①忘却ゲート(forget gate)でどの情報を破棄または保持するかを決定します。値は0から1の間で出てきます。0に近いほど忘れることを意味し、1に近いほど保持することを意味します。
2. 過去の状態を保持する機能を持つ、セル状態(cell state)が更新されます。
3. ②入力ゲート(input gate)は以前の隠れ状態と現在の入力をシグモイド関数に渡します。これにより、値を0と1の間で変換することによって更新される値が決まります。0は重要ではないことを意味し、1は重要であることを意味します。
4. ③出力ゲート(output gate)は、以前の隠れ状態と現在の入力をシグモイド関数に渡します。次に、新しく変更されたセル状態をtanh関数に渡します。tanhの出力とシグモイドの出力を乗算して、隠れ状態がどの情報を伝達するかを決定します。シグモイド関数によって、重要な情報であると判断すれば、更新されたセル状態の値を次に引き継ぐということです。

In [ ]:
embedding_layer = Embedding(word_vectors.shape[0],
                            word_vectors.shape[1],
                            weights=[word_vectors],
                            input_length=maxSeqLength,
                            trainable=False)
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)

opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# EaelyStoppingの設定
early_stopping =  EarlyStopping(
                            monitor='val_loss',
                            min_delta=0.0,
                            patience=2
    )

epochs = 100
batch_size = 16
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

LSTMモデルの精度を出力します。

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

## LSTMで分類クレームのテスト

モデルを構築したので、一連のクレームに対してそれを試してください。 最初にテキストを前処理する必要があることを思い出してください。

次のセルを実行して、テストデータを準備します。

In [ ]:
test_claim = ['I crashed my car into a pole.', 
              'The flood ruined my house.', 
              'I lost control of my car and fell in the river.']

test_claim_indices = convert_to_indices(test_claim, dictionary, contractions)
test_data = pad_sequences(test_claim_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

次に、モデルを使用して分類を予測します。

In [ ]:
pred = model.predict(test_data)
pred_label = pred.argmax(axis=1)
pred_df = pd.DataFrame(np.column_stack((pred,pred_label)), columns=['class_0', 'class_1', 'label'])
pred_df.label = pred_df.label.astype(int)
print('Predictions')
pred_df